In [1]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-qiawl_im/audiocraft_41d8cf03dd9348a9b5cd76a818b15aa7
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-qiawl_im/audiocraft_41d8cf03dd9348a9b5cd76a818b15aa7
  Resolved https://github.com/facebookresearch/audiocraft to commit c09cd0cb9075e15fc96e7c791d4ba6878079601c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.8 MB/s eta 0:00:00
  Preparing

Use MusicGen to generate music from text

In [2]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [3]:
model = musicgen.MusicGen.get_pretrained('medium', device='cuda')
model.set_generation_params(duration=8)

/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:112: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-medium
  warnings.warn(


In [6]:
SECOND = 4000

In [7]:
res = model.generate([
    """I'm doing very well! I'm so excited for my birthday!"""
],
    progress=True)
display_audio(res, 10 * SECOND)

In [8]:
res = model.generate([
    """I want to cry, why is this happening to me""",
],
    progress=True)
display_audio(res, 8 * SECOND)

Fine tune model using MusicLM and Transformers

In [9]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration


processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, max_new_tokens=256)

IPYNB ONLY: Show music

In [10]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)

DOWNLOAD

In [13]:
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].numpy())